## Analysis of cluster data produced by mPIXdaq
                                                    Günter Quast, Aug. 2025

This code analyzes the properties of pixel clusters derived from miniPIX frames 
uning the class `frameAnalyzer` of the Python package *mpixdaq*. The class 
is shown here:

``` 
Class frameAnalyzer: 
    Analyze frame data  

    - find clusters  
    - compute cluster energies  
    - compute position and covariance matrix of x- and y-coordinates  
    - analyze cluster shape (using eigenvalues of covariance matrix)  
    - construct a tuple with cluster properties  

    Note: this algorithm only works if clusters do not overlap!

    Args: a 2d-frame from the miniPIX

    Returns:

    - n_pixels: number of pixels with energy > 0
    - n_clusters: number of clusters
    - n_cpixels: number of pixels per cluster
    - circularity: circularity per cluster (0. for linear, 1. for circular)
    - cluster_energies: energy per cluster

    - self.clusters is a tuple with properties per cluster with mean of x and y coordinates,
       number of pixels, energy, eigenvalues of covariance matrix and orientation of the eigenvector
       corresponing to the largest eigenvalue in the range [-pi/2, pi/2].   
         format of hte tuple:
              ( (x,y), n_pix, energy, (var_mx, var_mn), angle )

```
These proberties per cluster or single, isolated pixel are exported to a *.csv* file with header   
   **x_mean** 	**y_mean** 	**n_pix** 	**energy** 	**var_mx** 	**var_mn** 	**angle**   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.colors import LogNorm

#### Import data and add derived quantities

In [ ]:
# import data in a pandas data frame
df = pd.read_csv("data/BlackForestStone_clusters.csv.gz")

# add derived quantities
#  - circularity defined as the ratio of the smaller and the larger eigenvalue 
#    of the covariance matrix of the pixels in a cluster
df['circularity'] = df['var_mn']/df['var_mx']

# show data
display(df)

#### Produce an overview of the distributions of the input variables

In [ ]:
def plt_keys(df):
    for key in df.keys():
        _ = plt.hist(df[key][df[key] != np.nan], bins=100, rwidth=0.85, edgecolor='grey')
        plt.yscale("log")
        plt.xlabel(key)
        plt.show()

plt_keys(df)

#### Spacial distribution of clusters

In [ ]:
_ = plt.hist2d(df['x_mean'], df['y_mean'], bins=(256, 256), norm=LogNorm())
plt.show()

#### 2d plots for different circulatities

In [ ]:
circularity_cut = 0.3
is_circular = df['circularity'] >= circularity_cut
is_linear = df['circularity'] < circularity_cut
is_single = df['n_pix'] <= 2

_ = plt.hist2d(df['energy'][is_circular], df['n_pix'][is_circular], bins = (100, 80), norm=LogNorm())
plt.xlim(0,10000)
plt.ylim(0,80)
plt.suptitle("circular")
plt.colorbar()
plt.xlabel("energy (keV)")
plt.ylabel("# pixels")
_ax = plt.gca()
_ax.text(0.8, 0.9, f"∑: {is_circular.sum()}", transform=_ax.transAxes, color='b') 
plt.show()

_ = plt.hist2d(df['energy'][is_linear], df['n_pix'][is_linear], bins = (100, 80), norm=LogNorm())
plt.xlim(0,10000)
plt.ylim(0,80)
plt.suptitle("linear")
plt.colorbar()
plt.xlabel("energy (keV)")
plt.ylabel("# pixels")
_ax = plt.gca()
_ax.text(0.8, 0.9, f"∑: {is_linear.sum()}", transform=_ax.transAxes, color='b') 
plt.show()

_ = plt.hist2d(df['energy'][is_single], df['n_pix'][is_single], bins = (100,10), norm=LogNorm()) 
plt.xlim(0,1000)
plt.ylim(0, 4)
plt.suptitle("single")
plt.colorbar()
plt.xlabel("energy (keV)")
plt.ylabel("# pixels")
_ax = plt.gca()
_ax.text(0.8, 0.9, f"∑: {is_single.sum()}", transform=_ax.transAxes, color='b') 
plt.show()


#### Discussion

These scatter plots show a very clear structure arising from the propeties of the detected particles.

  - alpha particles produce small, round clusters and deposit all of their energy in a small volume
  - beta particles produce long tracks in the material
  - photons procude - typically low-energy - electrons leading to signatures with
    very few or even isolated pixels

Note that some of the long tracks may have a significant "circularity" if they are bent !